In [ ]:
'''
This script prints the Figure 2 in Section IV of "Evaluation metrics and method for planned regulatory inspection targeting".
'''
__author__ = 'Celso Ribas'
__credits__ = ['Celso Ribas', 'José Bermudez']
__version__ = '0.2'
__maintainer__ = 'Celso Ribas'
__contact__ = 'celsoribas@gmail.com'
__status__ = 'dev'
__date__ = 'October 31, 2022'

In [ ]:
# loading libraries
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline

In [ ]:
# loading data
data = np.load('data_for_print_simulations.npz')
W = data['W']  # influences between municipalities
vertices = data['vertices']  # IBGE codes of municipalities
gdf = gpd.read_file("shapefile.shp")  # IBGE shapefile for municipalities in Figure 1.

In [ ]:
# INSERT THE CODE OF THE MUNICIPALITY OF INTEREST (from here onward, in comments, just vi)
code_mun = 3132404

In [ ]:
i = np.argwhere(vertices==code_mun)[0][0]  # index in W of vi

In [ ]:
def influences(i, direction):
    global W, vertices, gdf
    # setting variables
    if direction == 'in':
        wij = np.sort(W[i,:])[::-1]  # sorted influences from all municipalities on vi
        idx = np.argsort(W[i,:])[::-1]  # index of sorted influences
    elif direction == 'out':
        wij = np.sort(W[:,i])[::-1]  # sorted influences from all municipalities on vi
        idx = np.argsort(W[:,i])[::-1]  # index of sorted influences
    else:
        print('Direction must be "in" or "out"!')
    mask_wij = wij>0  # mask to influencing municipalities
    wij = wij[mask_wij]  # sorted influences just from influencing municipalities
    idx = idx[mask_wij]  # index of sorted influences just from influencing municipalities
    Wij = np.zeros([wij.size+1, wij.size+1])  # tensor for reduced adjacency matrix
    if direction == 'in':
        Wij[-1,:-1] = wij  # setting influences
    else:
        Wij[:-1,-1] = wij
    idx_Wij = np.append(idx,i)  # append the index of vi
    graph_Wij = nx.DiGraph(Wij.T)  # the adjacency matrix for NetworkX lib is transposed
    gdf_Wij = gpd.GeoDataFrame().reindex_like(gdf.loc[0:idx.size,:]) # GeoDataFrame
    # values of influencing and influenced municipalities
    for j in gdf_Wij.index:
        gdf_Wij.loc[j,:] = gdf.loc[gdf[gdf.CD_MUN == str(vertices[idx_Wij[j]])].index[0],:]
    # locations
    centroids_Wij = np.column_stack((gdf_Wij.centroid.x, gdf_Wij.centroid.y))  # centroids of municipalities
    positions_Wij = dict(zip(graph_Wij.nodes, centroids_Wij))  # positions of municipalities
    # Colors for edges
    map_edge = plt.get_cmap('cool')
    cNorm_Wij  = colors.Normalize(vmin=0, vmax=wij.max())
    scalarMap_Wij = cmx.ScalarMappable(norm=cNorm_Wij, cmap=map_edge)
    colorList_Wij = []
    for i in range(graph_Wij.number_of_edges()):
      colorVal_Wij = scalarMap_Wij.to_rgba(wij[i])
      colorList_Wij.append(colorVal_Wij)
    # Colors for municipalities
    map_mun = plt.get_cmap('binary')
    faceColorList = np.full((gdf_Wij.index.size,), 0.4)
    faceColorList[-1] = 1
    faceColorList = map_mun(faceColorList)
    
    return gdf_Wij, graph_Wij, positions_Wij, faceColorList, colorList_Wij, scalarMap_Wij

In [ ]:
# calcating necessary data for Figure
gdf_Wij, graph_Wij, positions_Wij, colorList_Vij, colorList_Wij, scalarMap_Wij = influences(i, 'in')
gdf_Wji, graph_Wji, positions_Wji, colorList_Vji, colorList_Wji, scalarMap_Wji = influences(i, 'out')

In [ ]:
ax_1 = gdf_Wij.plot(linewidth=.3, edgecolor="black", facecolor=colorList_Vij) # just to get axis

In [ ]:
sc = 5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
# first subplot
gdf.plot(linewidth=.3, ax = ax1, edgecolor="black", facecolor="white")
ax3 = gdf_Wij.plot(linewidth=.3, ax = ax1, edgecolor="black", facecolor=colorList_Vij)
nx.draw(graph_Wij, positions_Wij, width = 2, arrows=True, arrowsize = 4, ax=ax3, nodelist=[], node_size=1, node_color="white", edge_color=colorList_Wij)
ax1.axis([ax_1.axis()[0], ax_1.axis()[1], ax_1.axis()[2], ax_1.axis()[3]])
ax1.set_title('(a)', fontsize=15)
cbar_ij = fig.colorbar(scalarMap_Wij, ax = ax1)
# second subplot
gdf.plot(linewidth=.3, ax = ax2, edgecolor="black", facecolor="white")
ax4 = gdf_Wji.plot(linewidth=.3, ax = ax2, edgecolor="black", facecolor=colorList_Vji)
nx.draw(graph_Wji, positions_Wji, width = 2, arrows=True, arrowsize = 4, ax=ax4, nodelist=[], node_size=1, node_color="white", edge_color=colorList_Wji)
ax2.axis([ax_1.axis()[0], ax_1.axis()[1], ax_1.axis()[2], ax_1.axis()[3]])
ax2.set_title('(b)', fontsize=15)
cbar_ji = fig.colorbar(scalarMap_Wji, ax = ax2)
plt.savefig('Figure_1.png', format='png', dpi=120)
plt.show()